In [81]:
from io import BytesIO
import pycurl,timeit
from multiprocessing import Process
from multiprocessing import Manager

# Performance with proxy

In [82]:
def proxy_latency(prox_type,host,port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.PROXY, f"{host}:{port}")
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = prox_type
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    
    return m


# Performance without proxy

In [83]:
def normal_latency(port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = "N/A"
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    

# Setup

In [84]:
host = "130.64.148.78"
port = "8080"
range_k = 100



# target = "https://github.com/"

target = "http://www.cs.cmu.edu/~prs/bio.html"


# target = "http://www.cs.tufts.edu/comp/112/"

# Without proxy

## 1. One time

In [85]:
return_list = []
normal_latency(port,target,return_list)
print(return_list[0])

{'prox_type': 'N/A', 'target': 'http://www.cs.cmu.edu/~prs/bio.html', 'total-time': 0.077852, 'namelookup-time': 0.004268, 'connect-time': 0.02178, 'pretransfer-time': 0.021871, 'starttransfer-time': 0.042261}


## 2. Average time (100 request)

In [86]:
total = 0
return_list = []
for n in range(range_k):
    result = normal_latency(port,target,return_list)

for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.11252317999999999

# Our project - select + pthread

In [87]:
prox_type = "select"

## 1. One time without cache

In [88]:
return_list = []
proxy_latency(prox_type,host,port,target,return_list)
print(return_list[0])

{'prox_type': 'select', 'target': 'http://www.cs.cmu.edu/~prs/bio.html', 'total-time': 0.065777, 'namelookup-time': 5.1e-05, 'connect-time': 0.000165, 'pretransfer-time': 0.000219, 'starttransfer-time': 0.065711}


## 2. Average time with cache (100 reqests)

In [90]:
total = 0
return_list = []
for n in range(range_k):
    proxy_latency(prox_type,host,port,target,return_list)

    
for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.00046395000000000025

# Fully fork version - without cache

In [132]:
prox_type = "fork"


In [133]:
total = 0

for n in range(range_k):
    result = proxy_latency(prox_type,host,port,target)
    total+= result["total-time"]
    
total/range_k

0.07034185000000004

# Fully pthread version - without cache

In [134]:
prox_type = "pthread"

In [135]:
total = 0

for n in range(range_k):
    result = proxy_latency(prox_type,host,port,target)
    total+= result["total-time"]
    
total/range_k

0.11215383000000007

# Simulate multi users - multiprocesses

In [91]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=normal_latency,args=(port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k    

6.10859632

In [92]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=proxy_latency,args=(prox_type,host,port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k  

0.0007603799999999996